**Implemented By Muhammad Hanan Asghar**

In [ ]:
# Libraries needed for NLP
import nltk
nltk.download('punkt') # This Package is used to convert sentences to individual words
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# LancasterStemmer reduces the words by Making all to the root word
# Example: waiting, waited, waits are converted to root word 'wait'
# Example: cooker, cooks, cooked , cooking converted to root word 'cook'

!pip install tflearn
!pip install tensorflow
!pip install numpy

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [8]:
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],\n         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],\n         "context": [""]\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],\n         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],\n         "context": [""]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful", "Awesome, thanks", "Thanks for helping me"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"],\n         "context": [""]\n        },\n        {"tag": "noanswer",\n         "patterns": [],\n         "responses": ["Sorry, can\'t understand you", "Please give me more info", "Not sure I understand"],\n         "context"

In [9]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [10]:
intents

{'intents': [{'context': [''],
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'context': [''],
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'context': [''],
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'context': [''],
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'tag': 'noanswer'},
  {'context': [''],
   'patterns': ['How you could help me?',
    'What you can do?

In [12]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [13]:
words[:5] # This is Called Tokenization

['Hi', 'there', 'How', 'are', 'you']

In [14]:
documents[:5] # This is called Document Making in Chatbot Building.

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting')]

In [15]:
classes[:5] #This is called class seperating.

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug']

In [16]:
# Converting each word to their root word
__words = []
for word in words:
  if word not in ignore:
    __w = stemmer.stem(word)
    __words.append(__w.lower())

In [17]:
__words[:5] # This is called Lancaster Stemming Alghorithum

['hi', 'ther', 'how', 'ar', 'you']

In [18]:
# Let's Sort our __word list and removing duplicates
__words = sorted(list(set(__words)))

In [19]:
__words[:5]

["'s", ',', 'a', 'advers', 'al']

In [20]:
# Removing Duplicates by using set
classes = sorted(list(set(classes)))

In [21]:
classes[:5]

['adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'goodbye',
 'greeting']

In [22]:
print (len(documents), "Cocuments")
print (len(classes), "Classes", classes)
print (len(__words), "Unique Stemmed Words", __words)

47 Cocuments
9 Classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
84 Unique Stemmed Words ["'s", ',', 'a', 'advers', 'al', 'anyon', 'ar', 'awesom', 'be', 'behavy', 'blood', 'by', 'bye', 'can', 'caus', 'chat', 'check', 'could', 'dat', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hey', 'hi', 'hist', 'hol', 'hospit', 'how', 'i', 'id', 'is', 'lat', 'list', 'load', 'loc', 'log', 'look', 'lookup', 'man', 'me', 'mod', 'nearby', 'next', 'nic', 'of', 'off', 'op', 'paty', 'pharm', 'press', 'provid', 'react', 'rel', 'result', 'search', 'see', 'show', 'suit', 'support', 'task', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'transf', 'up', 'want', 'what', 'which', 'with', 'you']


#**Let's Convert our Preprocessed Data to Tensors**

In [23]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

In [24]:
# create training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # Stemming Our Doc Words
  __patterns = [stemmer.stem(word.lower()) for word in pattern_words if word not in ignore]
  # create bag of words array
  for w in words:
    bag.append(1) if w in __patterns else bag.append(0)

In [25]:
# create training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # Stemming Our Doc Words
  __patterns = [stemmer.stem(word.lower()) for word in pattern_words if word not in ignore]
  # create bag of words array
  for w in words:
    bag.append(1) if w in __patterns else bag.append(0)
  # output is '1' for current tag and '0' for rest of other tags
  output_row = list(output_empty)
  print(output_row)

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0

In [26]:
 # create training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # Stemming Our Doc Words
  __patterns = [stemmer.stem(word.lower()) for word in pattern_words if word not in ignore]
  # create bag of words array
  for w in words:
    bag.append(1) if w in __patterns else bag.append(0)
  # output is '1' for current tag and '0' for rest of other tags
  output_row = list(output_empty)
  # Here i am making target list for fully connected layer in Neural Network
  output_row[classes.index(doc[1])] = 1
  print(output_row)

[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0

In [27]:
 # create training set, bag of words for each sentence
for doc in documents:
  # initialize bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # Stemming Our Doc Words
  __patterns = [stemmer.stem(word.lower()) for word in pattern_words if word not in ignore]
  # create bag of words array
  for w in words:
    bag.append(1) if w in __patterns else bag.append(0)
  # output is '1' for current tag and '0' for rest of other tags
  output_row = list(output_empty)
  # Here i am making target list for fully connected layer in Neural Network
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

#**Theory Behind Bag of Words Approach**

To understand the bag of words approach, let's first start with the help of an example.

Suppose we have a corpus with three sentences:

    "I like to play football"
    "Did you go outside to play tennis"
    "John and I play tennis"

Now if we have to perform text classification, or any other task, on the above data using statistical techniques, we can not do so since statistical techniques work only with numbers. Therefore we need to convert these sentences into numbers.
Step 1: Tokenize the Sentences

The first step in this regard is to convert the sentences in our corpus into tokens or individual words. Look at the table below:
Sentence 1 	Sentence 2 	Sentence 3
I 	Did 	John
like 	you 	and
to 	go 	I
play 	outside 	play
football 	to 	tennis
	play 	
	tennis 	
Step 2: Create a Dictionary of Word Frequency

The next step is to create a dictionary that contains all the words in our corpus as keys and the frequency of the occurrence of the words as values. In other words, we need to create a histogram of the words in our corpus. Look at the following table:
```
Word 	Frequency
I 	2
like 	1
to 	2
play 	3
football 	1
Did 	1
you 	1
go 	1
outside 	1
tennis 	2
John 	1
and 	1
```
In the table above, you can see each word in our corpus along with its frequency of occurrence. For instance, you can see that since the word play occurs three times in the corpus (once in each sentence) its frequency is 3.

In our corpus, we only had three sentences, therefore it is easy for us to create a dictionary that contains all the words. In the real world scenarios, there will be millions of words in the dictionary. Some of the words will have a very small frequency. The words with very small frequency are not very useful, hence such words are removed. One way to remove the words with less frequency is to sort the word frequency dictionary in the decreasing order of the frequency and then filter the words having a frequency higher than a certain threshold.

Let's sort our word frequency dictionary:
```
Word 	Frequency
play 	3
tennis 	2
to 	2
I 	2
football 	1
Did 	1
you 	1
go 	1
outside 	1
like 	1
John 	1
and 	1
```
Step 3: Creating the Bag of Words Model

To create the bag of words model, we need to create a matrix where the columns correspond to the most frequent words in our dictionary where rows correspond to the document or sentences.

Suppose we filter the 8 most occurring words from our dictionary. Then the document frequency matrix will look like this:
```
	Play 	Tennis 	To 	I 	Football 	Did 	You 	go
Sentence 1 	1 	0 	1 	1 	1 	0 	0 	0
Sentence 2 	1 	1 	1 	0 	0 	1 	1 	1
Sentence 3 	1 	1 	0 	1 	0 	0 	0 	0
```
It is important to understand how the above matrix is created. In the above matrix, the first row corresponds to the first sentence. In the first, the word "play" occurs once, therefore we added 1 in the first column. The word in the second column is "Tennis", it doesn't occur in the first sentence, therefore we added a 0 in the second column for sentence 1. Similarly, in the second sentence, both the words "Play" and "Tennis" occur once, therefore we added 1 in the first two columns. However, in the fifth column, we add a 0, since the word "Football" doesn't occur in the second sentence. In this way, all the cells in the above matrix are filled with either 0 or 1, depending upon the occurrence of the word. Final matrix corresponds to the bag of words model.

In [28]:
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [30]:
len(train_x[0])

194

In [31]:
len(train_y[0])

9

**Creating Neural Network**

In [32]:


# resetting underlying graph data
from tensorflow.python.framework import ops
ops.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])]) # Input Layer and First Hidden Layer
net = tflearn.fully_connected(net, len(classes)) # Second Hidden Layer
net = tflearn.fully_connected(net, len(classes)) # Third Hidden Layer
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax') # Using Softmax in Output layer because this n eural network have more than 1 outputs
net = tflearn.regression(net) #The regression layer is used in TFLearn to apply a regression (linear or logistic) to the provided input.



# Defining model and setting up tensorboard (Deep Neural Network Model)
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**Training**

In [33]:
# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('chatbot.tflearn')

Training Step: 5999  | total loss: 0.40214 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.40214 - acc: 0.8422 -- iter: 40/47
Training Step: 6000  | total loss: 0.37906 | time: 0.035s
| Adam | epoch: 1000 | loss: 0.37906 - acc: 0.8437 -- iter: 47/47
--
INFO:tensorflow:/content/chatbot.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [34]:
# Let's Save our Model Structures
import pickle
with open( "training_data", "wb" ) as file:
  pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}
               , file)
file.close()

**Test Our Bot**

In [35]:
# restoring all the data structures
with open( "training_data", "rb" ) as file:
  data = pickle.load(file)
  words = data['words']
  classes = data['classes']
  train_x = data['train_x']
  train_y = data['train_y']
file.close()

In [36]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [37]:
# load the saved model
model.load('./chatbot.tflearn')

INFO:tensorflow:Restoring parameters from /content/chatbot.tflearn


In [38]:
def clean_up_sentence(sentence):
  # tokenizing the pattern
  sentence_words = nltk.word_tokenize(sentence)
  # stemming each word
  __sentence = []
  for word in sentence_words:
    __sentence.append(stemmer.stem(word.lower()))
  return __sentence

In [39]:
clean_up_sentence("HI How Are You")

['hi', 'how', 'ar', 'you']

In [40]:
# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for idx,wrd in enumerate(words):
            if wrd == s: 
                bag[idx] = 1
                if show_details:
                    print ("found in bag: %s" % wrd)

    return(np.array(bag))

In [41]:
# Error Threshold means chatbot only reply if less than threshold
ERROR_THRESHOLD = 0.30

In [42]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    __result = []
    for idx, r in enumerate(results):
      if r > ERROR_THRESHOLD:
        __result.append([idx, r])
    # sort by strength of probability
    __result.sort(key=lambda value: value[1], reverse=True)
    __pred = []
    for val in __result:
        __pred.append((classes[val[0]], val[1]))
    # return tuple of intent and probability
    return __pred

In [43]:
classify("How are you")

[('goodbye', 0.35115057), ('thanks', 0.32552552), ('greeting', 0.31122264)]

In [44]:
classify("Thanks")

[('greeting', 0.54171276)]

In [45]:
def response(sentence, userID='123', show_details=False):
  __result = classify(sentence)
    # if we have a classification then find the matching intent tag
  if __result:
  # loop as long as there are matches to process
    while __result:
      for i in intents['intents']:
        # find a tag matching the first result
        if i['tag'] == __result[0][0]:
          # a random response from the intent
          __bot_response = random.choice(i['responses'])
          return __bot_response
      __result.pop(0)

In [47]:
response("Thanks")

'Hello, thanks for asking'

In [84]:
# Let's Check line from training data
response("How are you", show_details=True)

"We're open every day 9am-9pm"

#**Adding Context Feature in Order to Save bot from wandering**

In [ ]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    __result = []
    for idx, r in enumerate(results):
      if r > ERROR_THRESHOLD:
        __result.append([idx, r])
    # sort by strength of probability
    __result.sort(key=lambda value: value[1], reverse=True)
    __pred = []
    for val in __result:
        __pred.append((classes[val[0]], val[1]))
    # return tuple of intent and probability
    return __pred
  
def response(sentence, auth='123', show_details=False):
  __result = classify(sentence)
    # if we have a classification then find the matching intent tag
  if __result:
  # loop as long as there are matches to process
    while __result:
      for i in intents['intents']:
        # find a tag matching the first result
        if i['tag'] == __result[0][0]:
          if "context" in i:
            if show_details:
              print("Context:", i['context'])
            context[auth] = i['context']
          if (not 'context_filter' in i) or (auth in context) and ('context_filter' in i) and (i['context_filter'] == context[auth]):
            if show_details:
              print("Tag:", i['tag'])
              # a random response from the intent
              __bot_response = random.choice(i['responses'])
              return __bot_response
      __result.pop(0)

In [ ]:
# Let's Check line from training data
response("Thanks")